#**Importing the necessary library**


In [1]:
import requests
import pandas as pd
import time

#**Extraction**

In [8]:
def extract_from_csv(file_path):
    """
    Reads a CSV file and returns it as a pandas DataFrame.

    Parameters:
        file_path (str): The path to the CSV file.

    Returns:
        pd.DataFrame: The loaded DataFrame.
    """
    # Read the CSV file using pandas
    df = pd.read_csv(file_path)
    # Return the DataFrame
    return df


##**Display Of the csv data**

In [9]:
data = extract_from_csv("all_countries.csv")
data.head()

,country,capital,region,continents,area,population,iso2,iso3
0,Lithuania,Vilnius,Europe,Europe,65300.0,2794700,LT,LTU
1,Chile,Santiago,Americas,South America,756102.0,19116209,CL,CHL
2,Benin,Porto-Novo,Africa,Africa,112622.0,12123198,BJ,BEN
3,Falkland Islands,Stanley,Americas,South America,12173.0,2563,FK,FLK
4,Georgia,Tbilisi,Asia,Asia,69700.0,3714000,GE,GEO


#**Extracting API**

In [7]:
def extract_from_api(api_url):
    """
    Extracts data from the World Bank API.
    Uses the per_page=300 trick to get all countries in one request.
    """
    print(f"Extracting data from API: {api_url}...")
    try:
        response = requests.get(api_url)
        response.raise_for_status() # Check for errors

        data = response.json()
        print("API data extracted successfully.")

        # The data we want is the SECOND item in the list
        country_list = data[1]

        # Return the list of countries
        return country_list

    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None

# --- To use this function ---

# The URL is all you need
url = 'https://api.worldbank.org/v2/country?format=json&per_page=300'

# Call the function ONCE
all_records = extract_from_api(url)

if all_records:
    print(f"Extraction Completed. Found {len(all_records)} records.")
    # Now you can transform all_records
    df = pd.json_normalize(all_records)
    print(df.head())

Extracting data from API: https://api.worldbank.org/v2/country?format=json&per_page=300...
API data extracted successfully.
Extraction Completed. Found 296 records.
    id iso2Code                         name capitalCity longitude latitude  \
0  ABW       AW                        Aruba  Oranjestad  -70.0167  12.5167   
1  AFE       ZH  Africa Eastern and Southern                                  
2  AFG       AF                  Afghanistan       Kabul   69.1761  34.5228   
3  AFR       A9                       Africa                                  
4  AFW       ZI   Africa Western and Central                                  

  region.id region.iso2code  \
0       LCN              ZJ   
1        NA              NA   
2       MEA              ZQ   
3        NA              NA   
4        NA              NA   

                                        region.value adminregion.id  \
0                         Latin America & Caribbean                   
1                              

In [18]:
def save_as_json(data, filename):
    """
    Task 4: Store the extracted (raw) data as json file.

    :param data: The JSON data (list or dict) to save.
    :param filename: The name of the file to save (e.g., 'raw_api_data.json').
    """
    print(f"Saving raw API data to {filename}...")
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print("Raw data saved successfully.")
    except IOError as e:
        print(f"Error saving JSON file: {e}")

**Transforming csv and printing it**

In [10]:
def transform_csv_data(df):
    """
    Task 2 (Part 1): Clean and transform the CSV dataset.

    Transforms the CSV DataFrame based on the 'all_countries.csv' file.
    - Selects 'country' and 'iso2' columns.
    - Renames 'country' to 'country_name' for merging.
    - Drops any rows with missing values.

    :param df: The raw DataFrame from the CSV.
    :return: A cleaned DataFrame or None if an error occurs.
    """
    print("Transforming CSV data...")
    try:
        # These are the correct columns from your uploaded file
        columns_to_keep = ['country', 'iso2']

        # Check if these columns exist
        if not all(col in df.columns for col in columns_to_keep):
            print("Error: The CSV file does not contain 'country' or 'iso2'.")
            print(f"Available columns are: {list(df.columns)}")
            return None

        transformed_df = df[columns_to_keep].copy()

        # Rename 'country' to 'country_name' to use as the standard merge key
        transformed_df = transformed_df.rename(columns={
            'country': 'country_name',
            'iso2': 'csv_iso2_code'
        })

        # Clean data by dropping rows where the country name is missing
        # Drop rows if EITHER 'country_name' OR 'csv_iso2_code' is missing
        transformed_df = transformed_df.dropna(subset=['country_name', 'csv_iso2_code'])

        print("CSV data transformed.")
        return transformed_df

    except Exception as e:
        print(f"Error transforming CSV data: {e}")
        return None

In [12]:
# 1. You load the data
my_data = extract_from_csv("all_countries.csv")

# 2. You run the transform function (BUT DON'T SAVE THE RESULT)
transform_csv_data(my_data)

# 3. You look at the original data... and it's unchanged!
print(my_data.head())

Transforming CSV data...
CSV data transformed.
            country     capital    region     continents      area  \
0         Lithuania     Vilnius    Europe         Europe   65300.0   
1             Chile    Santiago  Americas  South America  756102.0   
2             Benin  Porto-Novo    Africa         Africa  112622.0   
3  Falkland Islands     Stanley  Americas  South America   12173.0   
4           Georgia     Tbilisi      Asia           Asia   69700.0   

   population iso2 iso3  
0     2794700   LT  LTU  
1    19116209   CL  CHL  
2    12123198   BJ  BEN  
3        2563   FK  FLK  
4     3714000   GE  GEO  


In [15]:
# 1. Load the data
raw_df = extract_from_csv("all_countries.csv")

# 2. Run the function AND SAVE the new, returned copy
transformed_df = transform_csv_data(raw_df)

# 3. Print the NEW transformed data
print(transformed_df.head())

Transforming CSV data...
CSV data transformed.
       country_name csv_iso2_code
0         Lithuania            LT
1             Chile            CL
2             Benin            BJ
3  Falkland Islands            FK
4           Georgia            GE


In [17]:
transformed_df.head(10)

,country_name,csv_iso2_code
0,Lithuania,LT
1,Chile,CL
2,Benin,BJ
3,Falkland Islands,FK
4,Georgia,GE
5,Cyprus,CY
6,Ghana,GH
7,Belgium,BE
8,Cuba,CU
9,Andorra,AD


**Transforming Api And printing it**

In [19]:
def transform_api_data(raw_data):
    """
    Task 2 (Part 2): Clean and transform the API dataset.

    Transforms the raw API data from the World Bank.
    - Selects the second item in the list (the actual data).
    - Uses json_normalize to flatten the nested JSON.
    - Selects and renames relevant columns (name, incomeLevel.value, iso2Code).
    - Filters out 'Aggregates' (like 'Africa', 'World') to keep only countries.

    :param raw_data: The raw JSON data from the API.
    :return: A cleaned DataFrame or None if an error occurs.
    """
    print("Transforming API data...")

    # Check if data is valid
    if not raw_data or len(raw_data) < 2:
        print("Error: API data is empty or in an unexpected format.")
        return None

    try:
        # The data we want is the SECOND item in the list (index 1)
        # The first item (index 0) is just metadata
        country_list = raw_data[1]

        # Flatten the list of dictionaries into a DataFrame
        df = pd.json_normalize(country_list)

        # Select and rename the columns we want
        columns_to_keep = {
            'name': 'country_name',        # This is our merge key
            'iso2Code': 'api_iso2_code',
            'incomeLevel.value': 'income_level',    # This is our new data
            'region.value': 'region'            # We need this to filter
        }

        # Filter for only the columns we need
        transformed_df = df[columns_to_keep.keys()].copy()

        # Rename them to our standard names
        transformed_df = transformed_df.rename(columns=columns_to_keep)

        # IMPORTANT: Filter out "Aggregates"
        # The API returns rows for 'World', 'Africa', etc. We don't want them.
        transformed_df = transformed_df[transformed_df['region'] != 'Aggregates']

        # Drop region column as it's no longer needed
        transformed_df = transformed_df.drop(columns=['region'])

        print("API data transformed.")
        return transformed_df

    except KeyError:
        print("Error: The API data structure is wrong. Expected columns not found.")
        print(f"Available columns are: {list(df.columns)}")
        return None
    except Exception as e:
        print(f"Error transforming API data: {e}")
        return None

In [20]:
def merge_data(df_csv, df_api):
    """
    Task 3: Merge both datasets.

    Merges the two cleaned DataFrames on 'country_name'.

    :param df_csv: The cleaned DataFrame from all_countries.csv.
    :param df_api: The cleaned DataFrame from the World Bank API.
    :return: A merged DataFrame or None if an error occurs.
    """
    print("Merging data...")
    try:
        # Merge on the 'country_name' column
        # 'inner' merge keeps only countries that are present in BOTH datasets
        merged_df = pd.merge(df_csv, df_api, on='country_name', how='inner')

        print(f"Data merged successfully. {len(merged_df)} countries matched.")
        return merged_df
    except Exception as e:
        print(f"Error merging data: {e}")
        return None

In [21]:
def load_to_csv(df, output_filename):
    """
    Task 5: Store transformed data as csv file.

    :param df: The final merged DataFrame.
    :param output_filename: The name of the output file (e.g., 'transformed_data.csv').
    """
    print(f"Loading transformed data to {output_filename}...")
    try:
        df.to_csv(output_filename, index=False)
        print("Transformed data loaded successfully.")
    except IOError as e:
        print(f"Error saving final CSV file: {e}")

In [25]:
import requests
import pandas as pd
import json  # <-- This line is required
import os

def main():
    """
    Main function to run the entire ETL pipeline.
    """
    # Define file paths and URLs
    CSV_FILE = 'all_countries.csv'
    API_URL = 'https://api.worldbank.org/v2/country?format=json&per_page=300'
    RAW_JSON_FILE = 'raw_api_data.json'
    TRANSFORMED_CSV_FILE = 'transformed_country_data.csv'

    print("--- Starting ETL Pipeline ---")

    # EXTRACT
    raw_csv_df = extract_from_csv(CSV_FILE)
    if raw_csv_df is None:
        print("Halting pipeline due to CSV extraction error.")
        return

    raw_api_data = extract_from_api(API_URL)
    if raw_api_data is None:
        print("Halting pipeline due to API extraction error.")
        return

    # Task 4: Store raw extracted data
    save_as_json(raw_api_data, RAW_JSON_FILE)

    # TRANSFORM
    cleaned_csv_df = transform_csv_data(raw_csv_df)
    if cleaned_csv_df is None:
        print("Halting pipeline due to CSV transformation error.")
        return

    cleaned_api_df = transform_api_data(raw_api_data)
    if cleaned_api_df is None:
        print("Halting pipeline due to API transformation error.")
        return

    # LOAD (Merge & Save)
    # Task 3: Merge both datasets
    final_merged_df = merge_data(cleaned_csv_df, cleaned_api_df)
    if final_merged_df is None:
        print("Halting pipeline due to merge error.")
        return

    # Task 5: Store transformed data as CSV
    load_to_csv(final_merged_df, TRANSFORMED_CSV_FILE)

    print("\n--- ETL Pipeline Completed Successfully ---")
    print(f"Final merged data saved to: {TRANSFORMED_CSV_FILE}")
    print("\nFinal Data Preview (first 5 rows):")
    print(final_merged_df.head())

In [26]:
if __name__ == "__main__":
    main()

--- Starting ETL Pipeline ---
Extracting data from API: https://api.worldbank.org/v2/country?format=json&per_page=300...
API data extracted successfully.
Saving raw API data to raw_api_data.json...
Raw data saved successfully.
Transforming CSV data...
CSV data transformed.
Transforming API data...
API data transformed.
Merging data...
Data merged successfully. 0 countries matched.
Loading transformed data to transformed_country_data.csv...
Transformed data loaded successfully.

--- ETL Pipeline Completed Successfully ---
Final merged data saved to: transformed_country_data.csv

Final Data Preview (first 5 rows):
Empty DataFrame
Columns: [country_name, csv_iso2_code, api_iso2_code, income_level]
Index: []


In [29]:
df = pd.read_csv("transformed_country_data.csv")
df.head()

,country_name,csv_iso2_code,api_iso2_code,income_level
